<h1 align="center"> Исследование: Clickhouse vs Vertica </h1>

## 1. Подготовка кластера хранилища для работы

### `Clickhouse`

In [1]:
from clickhouse_driver import connect

clickhouse = connect(dsn='clickhouse://localhost:9000/default?user=default').cursor()

clickhouse.execute("""
    CREATE TABLE IF NOT EXISTS film_view_completed_events (
        timestamp TIMESTAMP,
        film_id   UUID,
        user_id   String,
        score UInt32,
    ) Engine=MergeTree() ORDER BY timestamp;
""")

The history saving thread hit an unexpected error (OperationalError('attempt to write a readonly database')).History will not be written to the database.


### `Vertica`

In [2]:
from vertica_python import connect

vertica = connect(dsn='vertica://localhost:5433/docker?user=dbadmin&autocommit=true').cursor()

vertica.execute("""
    CREATE TABLE IF NOT EXISTS film_view_completed_events (
        timestamp TIMESTAMP,
        film_id UUID,
        user_id   VARCHAR(100),
        score INTEGER
    ) ORDER BY timestamp;
""")

## 2. Скрипт для генерации данных

In [3]:
from faker import Faker

fake = Faker()


def gendata():
    return {
        'timestamp': fake.past_datetime(start_date='-1y'),
        'film_id': fake.uuid4(),
        'user_id': fake.uuid4(),
        'score': fake.random_int(max=100),

    }

## 3. Тестирование вставки данных (10 000 000 строк)

In [4]:
batch_size = 10000
multiplier = 1000

### `Clickhouse`

In [5]:
%%time
for _ in range(multiplier):
    clickhouse.execute(
        'INSERT INTO film_view_completed_events (timestamp, film_id, user_id, score) VALUES',
        [gendata() for _ in range(batch_size)],
    )

CPU times: user 2min 44s, sys: 455 ms, total: 2min 45s
Wall time: 3min 2s


In [6]:
clickhouse.execute('SELECT COUNT(*) FROM film_view_completed_events')
clickhouse.fetchone()

(10000000,)

### `Vertica`

In [7]:
%%time
for _ in range(multiplier):
    vertica.executemany(
        'INSERT INTO film_view_completed_events (timestamp, film_id, user_id, score) VALUES (:timestamp, :film_id, :user_id, :score)',
        [gendata() for _ in range(batch_size)],
    )

CPU times: user 3min 34s, sys: 1.33 s, total: 3min 35s
Wall time: 4min 11s


In [8]:
vertica.execute('SELECT COUNT(*) FROM film_view_completed_events')
vertica.fetchone()

[10000000]

## 4. Тестирование чтения данных (за 1 день)

In [9]:
from datetime import datetime, timedelta

yesterday = datetime.now() - timedelta(days=1)

### `Clickhouse`

In [10]:
%%time
clickhouse.execute(
    'SELECT * FROM film_view_completed_events WHERE timestamp  > %(datetime)s ORDER BY timestamp',
    {'datetime': yesterday},
)

CPU times: user 65.3 ms, sys: 5.61 ms, total: 70.9 ms
Wall time: 91.9 ms


In [11]:
clickhouse.fetchone()

(datetime.datetime(2024, 3, 8, 19, 59, 21),
 UUID('fd35439f-3508-4217-bdbf-a426395349fd'),
 '726f3447-bd4e-423d-95c9-9e1a29f6ba7b',
 62)

### `Vertica`

In [12]:
%%time
vertica.execute(
    'SELECT * FROM film_view_completed_events WHERE timestamp  > :datetime ORDER BY timestamp',
    {'datetime': yesterday},
)

CPU times: user 795 µs, sys: 427 µs, total: 1.22 ms
Wall time: 1.24 s


In [13]:
vertica.fetchone()

[datetime.datetime(2024, 3, 8, 19, 59, 19, 768863),
 UUID('58cbc238-eff0-4dc2-b7b3-46a57e186c29'),
 'bc0c8583-5a2c-4f08-bb6b-4991cecc4662',
 20]

## 5. Тестирование агрегации данных (фреймов)

### `Clickhouse`

In [14]:
%%time
clickhouse.execute('SELECT MAX(score), MIN(score), SUM(score), AVG(score) FROM film_view_completed_events')

CPU times: user 2.41 ms, sys: 440 µs, total: 2.85 ms
Wall time: 33.8 ms


In [15]:
clickhouse.fetchone()

(100, 0, 499912093, 49.9912093)

### `Vertica`

In [16]:
%%time
vertica.execute('SELECT MAX(score), MIN(score), SUM(score), AVG(score) FROM film_view_completed_events')

CPU times: user 55.1 ms, sys: 1.39 ms, total: 56.5 ms
Wall time: 230 ms


In [17]:
vertica.fetchone()

[100, 0, 499739740, 49.973974]

### **Вывод: ClickHouse быстрее справляется с аналитическими запросами, чем Vertica.**